
# Model Without Pipeline: 


In [3]:


import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

In [4]:

df = pd.read_csv("dataset/train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
175,176,0,3,"Klasen, Mr. Klas Albin",male,18.0,1,1,350404,7.8542,NaN,S
45,46,0,3,"Rogers, Mr. William John",male,NaN,0,0,S.C./A.4. 23567,8.0500,NaN,S
132,133,0,3,"Robins, Mrs. Alexander A (Grace Charity Laury)",female,47.0,1,0,A/5. 3337,14.5000,NaN,S
837,838,0,3,"Sirota, Mr. Maurice",male,NaN,0,0,392092,8.0500,NaN,S
516,517,1,2,"Lemore, Mrs. (Amelia Milley)",female,34.0,0,0,C.A. 34260,10.5000,F33,S


In [5]:

df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)
df.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
241,1,3,female,NaN,1,0,15.5,Q
740,1,1,male,NaN,0,0,30.0,S


In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df["Survived"],test_size=0.3)

print(f"X_train : {X_train.shape}")
print(f"X_test : {X_test.shape}")
print(f"y_train : {y_train.shape}")
print(f"y_test : {y_test.shape}")


X_train : (623, 7)
X_test : (268, 7)
y_train : (623,)
y_test : (268,)


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 623 entries, 691 to 751
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    623 non-null    int64  
 1   Sex       623 non-null    object 
 2   Age       487 non-null    float64
 3   SibSp     623 non-null    int64  
 4   Parch     623 non-null    int64  
 5   Fare      623 non-null    float64
 6   Embarked  622 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 38.9+ KB


In [8]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         136
SibSp         0
Parch         0
Fare          0
Embarked      1
dtype: int64

## We can't use column transformer efficiently if we don't know about sk-learn pipeline: 

`Here, it will give us error: `

- `In tnf2, we use the embarked column, and then the embaked turn into a array . When we use tnf3 for onehotEncoder to convert categorical data into it will not work:`


In [9]:

# in OneHotEncoder(handle_unknown="ignore") when we empty value in ["sex" and "embarked"]
# we toatally ignore this when our model get nan value while deypoling:


transformer = ColumnTransformer(
    transformers=[
        ("tnf1",SimpleImputer(strategy="mean"),["Age"]),
        ("tnf2",SimpleImputer(strategy="most_frequent"),["Embarked"]),
        ("tnf3",OneHotEncoder(sparse_output=False,drop="first",handle_unknown='ignore'),["Sex","Embarked"])
    ],
    remainder="passthrough",
)

In [10]:
X_train_transformed = transformer.fit_transform(X_train)

X_train_transformed

array([[4.0, 'C', 0.0, ..., 0.0, 1.0, 13.4167],
       [21.0, 'S', 0.0, ..., 1.0, 0.0, 9.825],
       [36.0, 'S', 1.0, ..., 1.0, 2.0, 120.0],
       ...,
       [27.0, 'S', 0.0, ..., 1.0, 0.0, 21.0],
       [40.0, 'S', 0.0, ..., 1.0, 1.0, 39.0],
       [6.0, 'S', 1.0, ..., 0.0, 1.0, 12.475]], dtype=object)

---

<br>
<br>
<br>


# Do all the process without using column transformer :


<br>
<br>
<br>

--- 


In [11]:
X_train.sample(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
764,3,male,16.0,0,0,7.775,S
814,3,male,30.5,0,0,8.050,S


In [12]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         136
SibSp         0
Parch         0
Fare          0
Embarked      1
dtype: int64

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 623 entries, 691 to 751
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    623 non-null    int64  
 1   Sex       623 non-null    object 
 2   Age       487 non-null    float64
 3   SibSp     623 non-null    int64  
 4   Parch     623 non-null    int64  
 5   Fare      623 non-null    float64
 6   Embarked  622 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 38.9+ KB



<br>

#### Remove missing value:

<br>


In [14]:
from sklearn.impute import SimpleImputer

si_age = SimpleImputer()
si_embarked =  SimpleImputer(strategy="most_frequent")

X_train_age = si_age.fit_transform(X_train[["Age"]])
X_test_age = si_age.transform(X_test[["Age"]])

X_train_Embarked = si_embarked.fit_transform(X_train[["Embarked"]])
X_test_Embarked = si_embarked.transform(X_test[["Embarked"]])



<br>

#### apply one hot encoding:

<br>


In [15]:

# after depoling model: if we give dataset where we have null value in sex then it will be ignored
one_sex = OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore") 
one_embarked = OneHotEncoder(drop="first",sparse_output=False,handle_unknown='ignore')

X_train_Sex = one_sex.fit_transform(X_train[["Sex"]])
X_test_sex = one_sex.transform(X_test[["Sex"]])

X_train_one_Embarked = one_embarked.fit_transform(X_train_Embarked)
X_test_one_Embarked = one_embarked.transform(X_test_Embarked)



In [23]:

X_train_rem = X_train.drop(columns=["Sex","Embarked","Age"])
X_test_rem = X_test.drop(columns=["Sex","Embarked","Age"])


In [17]:
print(X_test_age.shape)
print(X_test_sex.shape)
print(X_test_one_Embarked.shape)
print(X_test_rem.shape)

(268, 1)
(268, 1)
(268, 2)
(268, 4)


In [18]:

X_train_transformed = np.concatenate((X_train_age,X_train_Sex,X_train_one_Embarked,X_train_rem),axis=1)
X_test_taransformed = np.concatenate((X_test_age,X_test_sex,X_test_one_Embarked,X_test_rem),axis=1)


In [19]:
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)
y_prid = clf.predict(X_test_taransformed)

y_prid

array([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1])

In [20]:
print(len(y_test))
print(len(y_prid))

accuracy_score(y_test,y_prid) * 100

268
268


76.49253731343283

<br>
<br>
<br>
<br>

# Export the model:

<br>
<br>
<br>
<br>

In [21]:

import pickle

# When any one give categorical value then we need to convert it to numerical value
# That's why we also need the (one_embarked and one_sex)  object
# we skip the missing value:(To keep simple finaly we want to teach the sk-learn pipeline)


pickle.dump(one_embarked,open("model/without_pipeline_embarked",'wb'))
pickle.dump(one_sex,open("model/without_pipeline_sex.pkl",'wb'))
pickle.dump(clf,open('model/without_pipeline.pkl','wb'))
